**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [1]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io

import cv2
import json

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization, Input, Flatten

Using TensorFlow backend.


# MiniProject on Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [2]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [3]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

The function __act__ samples an action based on the current state $s$ we are in. It does according to the following rule : 

Let $p \in [0,1]$ be chosen uniformly.
$$
\text{act}(s) =
\left\{
    \begin{array}{ll}
        a_{rand} & \mbox{if } p \leq \epsilon \\
        \pi_{agent}(s) & \mbox{otherwise}
    \end{array}
\right.
$$
where $a_{rand}$ is an action chosen randomly (according to a uniform law) in the set of actions $\mathcal{A}$ and $\pi_{agent}$ is the policy of the agent at the moment the function __act__ is called.

__WHY IS $\epsilon$ ESSENTIAL ?__

$\epsilon$ allows the agent to explore the set of actions and thus the set of states even if its policy is restrictive (for example we could have $\forall s, \pi_{agent}(s) = a_0$)

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save() 
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [4]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [5]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=50 # set small when debugging
epochs_test=50 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

- The array __position__ allows us to keep in memory the cells of the board the rat already has visited.
- The array __board__ allows us to keep in memory the cheese/poisonous cells that are left to eat.

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [6]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        return np.random.randint(4)  #returns a random action in the 4 actions available

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [7]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        state = env.reset()                #As stated in "The Game" cell, we restart the game at each epoch
        win = 0
        lose = 0
        
        game_over = False
        
        while not game_over:                            #We perform the game as indicated in the "The Game" cell
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward
        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [8]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 10.5/15.0. Average score (-4.5)
Win/lose count 11.5/15.0. Average score (-4.0)
Win/lose count 11.5/13.0. Average score (-3.1666666666666665)
Win/lose count 14.5/17.0. Average score (-3.0)
Win/lose count 11.5/16.0. Average score (-3.3)
Win/lose count 10.5/16.0. Average score (-3.6666666666666665)
Win/lose count 11.0/17.0. Average score (-4.0)
Win/lose count 7.5/15.0. Average score (-4.4375)
Win/lose count 13.0/18.0. Average score (-4.5)
Win/lose count 13.5/17.0. Average score (-4.4)
Win/lose count 7.5/12.0. Average score (-4.409090909090909)
Win/lose count 10.0/14.0. Average score (-4.375)
Win/lose count 10.5/18.0. Average score (-4.615384615384615)
Win/lose count 11.5/10.0. Average score (-4.178571428571429)
Win/lose count 14.5/14.0. Average score (-3.8666666666666667)
Win/lose count 17.0/13.0. Average score (-3.375)
Win/lose count 6.5/6.0. Average score (-3.1470588235294117)
Win/lose count 7.5/9.0. Average score (-3.0555555555555554)
Win/lose count 6.5/10.0. Average sco

***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




__First point__

We have 
\begin{equation*}
\begin{split}
Q^{\pi}(s,a) & = E_{p^\pi}\left[\sum_{t\geq 0} \gamma^t r(s_t, a_t) |s_0 = s, a_0 = a \right] \\
& = E_{p^\pi}\left[r(s, a) + \gamma\sum_{t\geq 1} \gamma^{t-1} r(s_t, a_t) |s_0 = s, a_0 = a \right]
\end{split}
\end{equation*}

Yet,
\begin{equation*}
\begin{split}
E_{p^\pi}\left[\sum_{t\geq 1} \gamma^{t-1}r(s_t,a_t)|s_0 = s, a_0 = a \right] & = E_{p^\pi}\left[E_{p^\pi}\left[\sum_{t\geq 1} \gamma^{t-1}r(s_t,a_t)|s_1, a_1 \right]|s_0 = s, a_0 = a \right] \\
&= E_{p^\pi}\left[Q^\pi(s_1,a_1) |s_0, a_0 = a \right]
\end{split}
\end{equation*}
Thus
$$Q^{\pi}(s,a)=E_{(s',a')\sim p^\pi(.,.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]$$

__Second point__

__Remark__ : $\pi^*$ is a probability on the set of actions, not the set of states, the following result holds for $s' \sim p(.|s,a)$ and not $ s' \sim \pi^*(.|s,a)$

Suppose there exists $\pi_{max}$ such that
$$ a \in \text{argmax}_{a'} Q^*(s,a') \iff \pi_{max}(a|s) > 0 $$
For such a policy we have 
$$E_{a \sim \pi}\left[Q^*(s,a)\right]= \max_a Q^*(s,a)$$
Besides, for $(s_0, a_0)$,

\begin{equation*}
\begin{split}
Q^*(s_0,a_0) & = E_{(s_1,a_1)\sim p^{\pi^*}(.,.|s_0,a_0)}[r(s_0,a_0)+\gamma Q^{\pi}(s_1,a_1)] \\
& \leq E_{(s_1,a_1)\sim p(.|s_0,a_0)}[r(s_0,a_0)+\gamma \max_{a_1}Q^{\pi}(s_1,a_1)] \\
& = E_{(s_1,a_1)\sim p^{\pi_{max}}(.,.|s_0,a_0)}[r(s_0,a_0)+\gamma Q^{\pi}(s_1,a_1)] \\
& \leq ... \\
& \leq E_{p^{\pi_{max}}}\left[\sum_{t\geq 0} r(s_t,a_t)\right] \\
& = Q^{\pi_{max}}(s_0,a_0)
\end{split}
\end{equation*}

By definition of $Q^*$, we also have $Q^* \geq Q^{\pi_{max}}$. Thus, $Q^* = Q^{\pi_{max}}$. We deduce

$$Q^{*}(s,a)=E_{s'\sim p(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')]$$

__Last point__

The previous point can be rewritten as 

$$E_{s'\sim p(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a') - Q^{*}(s,a)] = 0$$

It is therefore natural to want to minimize the objective 

$$\mathcal{L}(\theta)=E_{s' \sim p(.|s,a)}\Vert r+\gamma\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}$$

***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [9]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        self.memory.append(m)
        if len(self.memory) > self.max_memory:          #if the memory is exceeded, we delete its oldest element
            del self.memory[0]
            

    def random_access(self):
        arg = np.random.randint(len(self.memory))           #we choose a random argument
        return self.memory[arg]

***
The pipeline we will use for training is given below:

In [10]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [11]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        s_flat = s[None]
        pred = self.model.predict(s_flat)                     #We compute the values of the model at the current state and then sample 
        return np.argmax(pred)                             # the action that maximizes it

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            s_, n_s_, a_, r_, game_over_ = self.memory.random_access()        #We take randomly an element from the memory
            target_q[i] =self.model.predict(s_.reshape([1,s_.shape[0],s_.shape[1],s_.shape[2]]))[0]
                                                                  
            if game_over_:
                input_states[i] = s_
                target_q[i,a_] = r_
            else:
                input_states[i] = s_
                target_q[i,a_] = r_  + self.discount*max(self.model.predict(n_s_.reshape([1,s_.shape[0],s_.shape[1],s_.shape[2]]))[0])
        
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)

        l = self.model.train_on_batch(input_states, target_q)


        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
        model = Sequential()
        model.add(Flatten(input_shape=(5,5,self.n_state,)))
        model.add(Dense(32, activation = 'relu')) 
        model.add(Dense(4))

        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model
   ## With 50 epochs, we seem to be learning     

In [12]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train10.mp4'))

Epoch 000/050 | Loss 0.0008 | Win/lose count 0/3.0 (-3.0)
Epoch 001/050 | Loss 0.0004 | Win/lose count 3.5/5.0 (-1.5)
Epoch 002/050 | Loss 0.0043 | Win/lose count 1.5/1.0 (0.5)
Epoch 003/050 | Loss 0.0030 | Win/lose count 4.5/1.0 (3.5)
Epoch 004/050 | Loss 0.0023 | Win/lose count 9.0/12.0 (-3.0)
Epoch 005/050 | Loss 0.0081 | Win/lose count 4.5/5.0 (-0.5)
Epoch 006/050 | Loss 0.0055 | Win/lose count 1.5/2.0 (-0.5)
Epoch 007/050 | Loss 0.0167 | Win/lose count 3.5/6.0 (-2.5)
Epoch 008/050 | Loss 0.0303 | Win/lose count 8.0/6.0 (2.0)
Epoch 009/050 | Loss 0.0252 | Win/lose count 7.5/2.0 (5.5)
Epoch 010/050 | Loss 0.0185 | Win/lose count 3.0/4.0 (-1.0)
Epoch 011/050 | Loss 0.0108 | Win/lose count 1.5/0 (1.5)
Epoch 012/050 | Loss 0.0112 | Win/lose count 1.5/1.0 (0.5)
Epoch 013/050 | Loss 0.0689 | Win/lose count 7.5/5.0 (2.5)
Epoch 014/050 | Loss 0.0583 | Win/lose count 3.0/3.0 (0.0)
Epoch 015/050 | Loss 0.0025 | Win/lose count 8.0/2.0 (6.0)
Epoch 016/050 | Loss 0.0071 | Win/lose count 2.0/0 (

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [24]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        model = Sequential()
        model.add(Conv2D(50,(2,2),input_shape=(5,5,self.n_state,),activation='relu'))
        model.add(Conv2D(25,(2,2),activation='relu'))
        model.add(Flatten())
        model.add(Dense(4))
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [25]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent,env,epochs_train,prefix='cnn_train')
HTML(display_videos('cnn_train10.mp4'))

Epoch 000/050 | Loss 0.0030 | Win/lose count 2.0/4.0 (-2.0)
Epoch 001/050 | Loss 0.0032 | Win/lose count 3.0/4.0 (-1.0)
Epoch 002/050 | Loss 0.0504 | Win/lose count 4.5/5.0 (-0.5)
Epoch 003/050 | Loss 0.0033 | Win/lose count 3.0/6.0 (-3.0)
Epoch 004/050 | Loss 0.0104 | Win/lose count 3.5/4.0 (-0.5)
Epoch 005/050 | Loss 0.0058 | Win/lose count 6.5/5.0 (1.5)
Epoch 006/050 | Loss 0.0019 | Win/lose count 4.0/3.0 (1.0)
Epoch 007/050 | Loss 0.0062 | Win/lose count 3.0/1.0 (2.0)
Epoch 008/050 | Loss 0.0081 | Win/lose count 3.0/3.0 (0.0)
Epoch 009/050 | Loss 0.0087 | Win/lose count 12.0/3.0 (9.0)
Epoch 010/050 | Loss 0.0022 | Win/lose count 10.5/0 (10.5)
Epoch 011/050 | Loss 0.0028 | Win/lose count 8.5/4.0 (4.5)
Epoch 012/050 | Loss 0.0043 | Win/lose count 9.5/2.0 (7.5)
Epoch 013/050 | Loss 0.0025 | Win/lose count 8.0/3.0 (5.0)
Epoch 014/050 | Loss 0.0037 | Win/lose count 14.5/4.0 (10.5)
Epoch 015/050 | Loss 0.0029 | Win/lose count 12.5/3.0 (9.5)
Epoch 016/050 | Loss 0.0037 | Win/lose count 4.

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [20]:
env = Environment(grid_size=size, max_time=T,temperature=0.5)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=1000, batch_size = 64)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=1000, batch_size = 64)
agent_fc.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
Win/lose count 19.5/6.0. Average score (13.5)
Win/lose count 26.0/3.0. Average score (18.25)
Win/lose count 20.0/1.0. Average score (18.5)
Win/lose count 24.0/4.0. Average score (18.875)
Win/lose count 15.5/2.0. Average score (17.8)
Win/lose count 28.0/3.0. Average score (19.0)
Win/lose count 28.0/2.0. Average score (20.0)
Win/lose count 5.0/4.0. Average score (17.625)
Win/lose count 23.5/4.0. Average score (17.833333333333332)
Win/lose count 17.0/3.0. Average score (17.45)
Win/lose count 21.5/3.0. Average score (17.545454545454547)
Win/lose count 20.5/2.0. Average score (17.625)
Win/lose count 8.0/2.0. Average score (16.73076923076923)
Win/lose count 12.0/4.0. Average score (16.107142857142858)
Win/lose count 13.5/2.0. Average score (15.8)
Win/lose count 37.5/1.0. Average score (17.09375)
Win/lose count 22.5/4.0. Average score (17.176470588235293)
Win/lose count 20.5/5.0. Average score (17.083333333333332)
Win/lose count 19.0/1.0. Average score (17.13157894736842)
Win/

In [16]:
HTML(display_videos('cnn_test10.mp4'))

In [17]:
HTML(display_videos('fc_test10.mp4'))

- With both algorithms, the agent does not explore the set of states really widely with the parameter $\epsilon = 0.1$. We could take a greater $\epsilon$ to explore more.
- with a temperature equal to 0.3 the scores are pretty bad, by increasing the temperature we get much better result. However by increasing the temperature, the game gets easier (there are less poisonous cells and more cheese cells)
- No matter what the temperature is, FC seems to give a better score than CNN

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [21]:
def train_explore(agent,env,epoch,epsilon_decrease_rate=0.955,prefix=''):
    # Number of won games
    score = 0
    loss = 0
    
    for e in range(epoch):
        agent.set_epsilon((-0.5/epoch)*e + 0.5)
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0
        
        while not game_over:
            
            # exploration 
            agent.set_epsilon(agent.epsilon*epsilon_decrease_rate)
            
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action, True)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose - reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)
        
         # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
    
    
        
class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature
        
        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))
        self.malus_position = np.zeros((grid_size,grid_size))
        
        # coordinate of the cat
        self.x = 0
        self.y = 1
        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))
        
    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)
    
    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b
    
    def act(self, action,train=False):
        """This function returns the new state, reward and decides if the
        game ends."""
        
        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
    
        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')
        
        self.t = self.t + 1
        reward = 0
        
        if train:
            reward = -self.malus_position[self.x, self.y]
        self.malus_position[self.x, self.y] = 0.1
        reward += self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]
        
        return state, reward, game_over
    
    def reset(self):
        """This function resets the game and returns the initial state"""
    
        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]

        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))
        
        malus[bonus>0]=0

        self.board = bonus + malus
        
        self.board = bonus + malus
        self.malus_position = np.zeros((self.grid_size,self.grid_size))
        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0
        
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        
        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state
    
## use those samples of code:
#In train explore:
#state, reward, game_over = env.act(action, train=True)

## In Environment exploring:
# You will have to change n_state to 3 because you will use one more layer!
#reward = 0
#if train:
#    reward = -self.malus_position[self.x, self.y]
#self.malus_position[self.x, self.y] = 0.1

#reward = reward + self.board[self.x, self.y]
# 3 "feature" states instead of 2
#state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
#                                self.board.reshape(self.grid_size, self.grid_size,1),
#                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

In [22]:
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32,n_state=3)
train_explore(agent, env, epochs_train, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore10.mp4'))

Epoch 000/050 | Loss 0.0120 | Win/lose count 8.0/24.50000000000008 (-16.50000000000008)
Epoch 001/050 | Loss 0.0044 | Win/lose count 6.0/18.399999999999995 (-12.399999999999995)
Epoch 002/050 | Loss 0.0075 | Win/lose count 7.0/21.700000000000042 (-14.700000000000042)
Epoch 003/050 | Loss 0.0066 | Win/lose count 6.0/17.599999999999984 (-11.599999999999984)
Epoch 004/050 | Loss 0.0043 | Win/lose count 9.5/18.699999999999996 (-9.199999999999996)
Epoch 005/050 | Loss 0.0040 | Win/lose count 1.0/20.10000000000002 (-19.10000000000002)
Epoch 006/050 | Loss 0.0067 | Win/lose count 4.5/19.600000000000012 (-15.100000000000012)
Epoch 007/050 | Loss 0.0070 | Win/lose count 16.5/14.299999999999965 (2.200000000000035)
Epoch 008/050 | Loss 0.0045 | Win/lose count 14.5/14.599999999999964 (-0.09999999999996412)
Epoch 009/050 | Loss 0.0023 | Win/lose count 8.0/19.700000000000014 (-11.700000000000014)
Epoch 010/050 | Loss 0.0274 | Win/lose count 11.5/15.69999999999996 (-4.19999999999996)
Epoch 011/050 | 

In [23]:
# Evaluation
test(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore10.mp4'))

Win/lose count 17.5/4.0. Average score (13.5)
Win/lose count 13.5/0. Average score (13.5)
Win/lose count 4.5/0. Average score (10.5)
Win/lose count 19.0/3.0. Average score (11.875)
Win/lose count 12.0/0. Average score (11.9)
Win/lose count 6.5/1.0. Average score (10.833333333333334)
Win/lose count 10.0/1.0. Average score (10.571428571428571)
Win/lose count 2.5/0. Average score (9.5625)
Win/lose count 6.0/0. Average score (9.166666666666666)
Win/lose count 4.0/0. Average score (8.65)
Win/lose count 16.0/0. Average score (9.318181818181818)
Win/lose count 19.0/2.0. Average score (9.958333333333334)
Win/lose count 24.5/0. Average score (11.076923076923077)
Win/lose count 23.5/3.0. Average score (11.75)
Win/lose count 9.0/0. Average score (11.566666666666666)
Win/lose count 2.0/0. Average score (10.96875)
Win/lose count 7.0/0. Average score (10.735294117647058)
Win/lose count 5.0/0. Average score (10.416666666666666)
Win/lose count 13.5/0. Average score (10.578947368421053)
Win/lose count 

***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

***